<a href="https://colab.research.google.com/github/Colley-K/Time_series_forecasting/blob/master/clean_up_grocery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Clean Up Report

​



## 1. Setup & Import 

In [0]:
#Colab stuffs:

#get a fast operator system
!nvidia-smi

#mount google drive
from google.colab import drive
drive.mount('/content/drive')#click on the link it provides and copy and paste that code into the authorization area

#access the OS system to work with current directories:
import os 

Wed Nov  6 13:53:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
sns.set_style("whitegrid")
sns.despine()
sns.set(rc={'figure.figsize':(15,9)})
%matplotlib inline

In [0]:
#Turn the CSV files into data frames
armast = pd.read_csv('drive/My Drive/Capstone_2/data/armast.csv')  
artran = pd.read_csv('drive/My Drive/Capstone_2/data/artran.csv') 
armast2 = pd.read_excel ('drive/My Drive/Capstone_2/data/new_data.xlsx', 'THY_armast')
artran2 = pd.read_excel ('drive/My Drive/Capstone_2/data/new_data.xlsx', 'THY_artran')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##2. Data Cleanup-ARMAST


*   Columns to delete because no added value: 
>* editedby, addedby, edited, pickup (only two dates present)
*   Columns to delete because too many null values:
>* 'cshipno', 'puller', 'ponum', 'invtype'




In [0]:
armast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72561 entries, 0 to 72560
Data columns (total 24 columns):
invno       72561 non-null object
custno      72561 non-null object
cshipno     31002 non-null object
invdate     72561 non-null object
amount      72561 non-null float64
paid        72561 non-null float64
status      72561 non-null object
duedate     72561 non-null object
days        72561 non-null int64
sono        72522 non-null float64
delivery    72561 non-null object
ponum       15804 non-null object
addedby     72561 non-null object
editedby    72561 non-null object
added       72561 non-null object
edited      72561 non-null object
route       69588 non-null object
seq         63558 non-null object
salesman    70666 non-null object
priccode    72522 non-null object
invtype     12334 non-null object
firstrun    72561 non-null int64
pickup      72561 non-null object
puller      8760 non-null float64
dtypes: float64(4), int64(2), object(18)
memory usage: 13.3+ MB


In [0]:
armast.isnull().sum()

invno           0
custno          0
cshipno     41559
invdate         0
amount          0
paid            0
status          0
duedate         0
days            0
sono           39
delivery        0
ponum       56757
addedby         0
editedby        0
added           0
edited          0
route        2973
seq          9003
salesman     1895
priccode       39
invtype     60227
firstrun        0
pickup          0
puller      63801
dtype: int64

### 2.1 DROPPING USELESS COLUMNS





In [0]:
armast= armast.drop(columns= ['cshipno', 'puller', 'ponum', 'editedby', 'addedby', 'edited', 'invtype', 'pickup'])
armast2= armast2.drop(columns= ['cshipno', 'puller', 'ponum', 'editedby', 'addedby', 'edited', 'invtype', 'pickup'])


### 2.2 CHANGING DATA TYPES

In [0]:
#Convert all date strings to date time objects
from datetime import datetime

cols= ['invdate', 'duedate', 'delivery']
for col in cols:
  armast[col] =  pd.to_datetime(armast[col], format='%m/%d/%Y')

for col in cols:
  armast2[col] =  pd.to_datetime(armast2[col], format='%m/%d/%Y')


In [0]:
#Changing invoice number to int64 and converting all letter numbers into NaN's--- this will help us match up the invoice numbers and merge the two dataframes
armast.invno = pd.to_numeric(armast.invno, errors='coerce', downcast='float')
armast2.invno = pd.to_numeric(armast2.invno, errors='coerce', downcast='float')

### 2.3 EXPLORING COLUMN DATA


*   custno could be a a useful target predictor, or way to filter the data
*   What's going on with inventory type column? I want this column to be good!
*   Invno, must be kept as a string because some have letters in them
*   First run, route, salesman, status all could be one-hot coded...


In [0]:
print ("Column Descriptions: \n")
for col in armast.columns:
  print (f"{col} column:")
  print (f"Data Type: {armast[col].dtypes}")
  print (f"Unique Vals: {armast[col].nunique()}")
  print (f"Nulls: {armast[col].isnull().sum()} \n")

Column Descriptions: 

invno column:
Data Type: float32
Unique Vals: 72522
Nulls: 39 

custno column:
Data Type: object
Unique Vals: 613
Nulls: 0 

invdate column:
Data Type: datetime64[ns]
Unique Vals: 944
Nulls: 0 

amount column:
Data Type: float64
Unique Vals: 24823
Nulls: 0 

paid column:
Data Type: float64
Unique Vals: 24361
Nulls: 0 

status column:
Data Type: object
Unique Vals: 3
Nulls: 0 

duedate column:
Data Type: datetime64[ns]
Unique Vals: 992
Nulls: 0 

days column:
Data Type: int64
Unique Vals: 9
Nulls: 0 

sono column:
Data Type: float64
Unique Vals: 65070
Nulls: 39 

delivery column:
Data Type: datetime64[ns]
Unique Vals: 944
Nulls: 0 

added column:
Data Type: object
Unique Vals: 968
Nulls: 0 

route column:
Data Type: object
Unique Vals: 18
Nulls: 2973 

seq column:
Data Type: object
Unique Vals: 110
Nulls: 9003 

salesman column:
Data Type: object
Unique Vals: 15
Nulls: 1895 

priccode column:
Data Type: object
Unique Vals: 18
Nulls: 39 

firstrun column:
Data Type

In [0]:
#I'm curious how many of the 72K entries over and underpaid their bill amount
armast['diff_amt_paid'] = armast.amount- armast.paid
print (f"Number of entries that overpaid bill amount: {armast[armast.diff_amt_paid > 0.0].shape[0]}")
print (f"Number of entries that underpaid bill amount: {armast[armast.diff_amt_paid < 0.0].shape[0]}")
print (f"Number of entries that paid in full(pefect payment): {armast[armast.diff_amt_paid == 0.0].shape[0]}")
    

Number of entries that overpaid bill amount: 1601
Number of entries that underpaid bill amount: 643
Number of entries that paid in full(pefect payment): 70317


In [0]:
#Does this correspond to the status types?? 
armast.status.value_counts()#No, it looks like status types are independent of this...

X    65095
V     5232
O     2234
Name: status, dtype: int64

## 3. Data Cleanup-ARTRAN 

In [0]:
artran.head()

,sono,invno,invline,item,desc,uom,units,ordqty,shipqty,price,totalcost,notes,datesold,orgprice,cool,coolprnt
0,211277,207305,1,30172,"Carrot, Table, 25lb, Cert. OG",CASE,1.0,4.0,4.0,26.50,20.00,NaN,1/2/2017,26.50,NaN,US
1,211279,207306,1,40009C,"Tomato, 1 Layer 25ct 15lb, Certifie",CASE,1.0,1.0,1.0,49.90,34.00,NaN,1/2/2017,49.90,NaN,US
2,211280,207307,1,20081,"Kale, Lacinato 24ct, Certified Orga",CASE,1.0,1.0,1.0,30.90,24.00,NaN,1/2/2017,30.90,NaN,US
3,211285,207308,1,CV70187A,"Juice, Apple,1 gallon NATURAL, LOCA",EACH,1.0,4.0,4.0,4.90,3.75,NaN,1/2/2017,4.90,NaN,US
4,211285,207308,2,G5103,"Jam, Blackberry 9 lb, Certified Org",EACH,1.0,1.0,1.0,49.35,39.50,NaN,1/2/2017,49.35,NaN,US


In [0]:
artran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528748 entries, 0 to 528747
Data columns (total 16 columns):
sono         528748 non-null int64
invno        528748 non-null int64
invline      528748 non-null int64
item         528747 non-null object
desc         528748 non-null object
uom          528747 non-null object
units        528748 non-null float64
ordqty       528748 non-null float64
shipqty      528748 non-null float64
price        528748 non-null float64
totalcost    528748 non-null float64
notes        3853 non-null object
datesold     528748 non-null object
orgprice     528748 non-null float64
cool         14328 non-null object
coolprnt     361988 non-null object
dtypes: float64(6), int64(3), object(7)
memory usage: 64.5+ MB


In [0]:
artran.isnull().sum()

sono              0
invno             0
invline           0
item              1
desc              0
uom               1
units             0
ordqty            0
shipqty           0
price             0
totalcost         0
notes        524895
datesold          0
orgprice          0
cool         514420
coolprnt     166760
dtype: int64

### 3.1 DROPPING USELESS COLUMN

In [0]:
#Dropping columns with too many NaNs

artran.drop(columns= ['notes', 'cool', 'coolprnt'], inplace= True)
artran2.drop(columns= ['notes', 'cool', 'coolprnt'], inplace= True)

### 3.2 CHANGING DATA TYPES

In [0]:
#Convert all date strings to date time objects
from datetime import datetime

cols= ['datesold']
def change_date_time(df, cols=None):
  for col in cols:
    df[col] =  pd.to_datetime(df[col], format='%m/%d/%Y')

change_date_time(artran, cols)
change_date_time(artran2, cols)

### 3.3 EXPLORING COLUMN DATA

In [0]:
def col_descriptors (df):
  print ("Column Descriptions: \n")
  for col in df.columns:
    print (f"{col} column:")
    print (f"Data Type: {df[col].dtypes}")
    print (f"Unique Vals: {df[col].nunique()}")
    print (f"Nulls: {df[col].isnull().sum()} \n")
    
col_descriptors(artran)

Column Descriptions: 

sono column:
Data Type: int64
Unique Vals: 60577
Nulls: 0 

invno column:
Data Type: int64
Unique Vals: 67396
Nulls: 0 

invline column:
Data Type: int64
Unique Vals: 443
Nulls: 0 

item column:
Data Type: object
Unique Vals: 3088
Nulls: 1 

desc column:
Data Type: object
Unique Vals: 4097
Nulls: 0 

uom column:
Data Type: object
Unique Vals: 41
Nulls: 1 

units column:
Data Type: float64
Unique Vals: 41
Nulls: 0 

ordqty column:
Data Type: float64
Unique Vals: 449
Nulls: 0 

shipqty column:
Data Type: float64
Unique Vals: 543
Nulls: 0 

price column:
Data Type: float64
Unique Vals: 1597
Nulls: 0 

totalcost column:
Data Type: float64
Unique Vals: 1728
Nulls: 0 

datesold column:
Data Type: datetime64[ns]
Unique Vals: 955
Nulls: 0 

orgprice column:
Data Type: float64
Unique Vals: 635
Nulls: 0 



### 3.4 *Least amount of items ordered*
* There are about 274 items that were only ordered once
* 152 items that were ordered twice
* 132 items that were only ordered three times
* In total these items on account for .1 percent of all the invoices





In [0]:
558/artran.shape[0]*100 #calculating the percentage of items ordered under a certain amount
item_count = artran.groupby(['item']).item.agg('count').to_frame('count').reset_index() #Counting all the items to see which were ordered least and most
item_count.sort_values(by='count')


,item,count
2544,CV20384A,1
1970,70016CV,1
2645,CV20432J,1
209,10139A,1
662,10490B,1
207,10138B,1
2643,CV20432H,1
2646,CV20432K,1
660,10489W,1
868,20213F,1


### 3.5 FEATURE ENGINEERING

In [0]:
#adding columns

#Adding a column that detects a difference between order quantity and shipped quantity (not having enough stock)
artran['diff_quantity'] = artran['ordqty'] - artran['shipqty']




In [0]:
print ("How many times are they understocked for their orders?")
print (artran[artran.diff_quantity > 0].shape[0])

print ("any time overshipped?")
print (artran[artran.diff_quantity < 0].shape[0])

print ("how many times perfect quantity?")
print (artran[artran.diff_quantity == 0].shape[0])

How many times are they understocked for their orders?
45111
any time overshipped?
326
how many times perfect quantity?
483311


## 4. MERGING THE TWO DATASETS

In [0]:
#Function to figure out how many rows we are loosing

merge_outer_invo = pd.merge(armast, artran, on=['invno'], how = "outer")
print (f"Outer Merge on invoice number. \n Number of rows: {merge_outer_invo.shape[0]}\n")

left_armast= pd.merge(armast, artran, on ="invno", how= "left")
print (f"Merging on ARMAST.invno. \n Number of rows: {left_armast.shape[0]}\n")

right_artran= pd.merge(armast, artran, on ="invno", how= "right")
print (f"Merging on ARTRAN.invno. \n Number of rows: {right_artran.shape[0]}\n")

####################################

merge_outer_sono = pd.merge(armast, artran, on=['sono'], how = "outer")
print (f"Outer Merge on sono number. \n Number of rows: {merge_outer_sono.shape[0]}\n")

left_armast_s= pd.merge(armast, artran, on ="sono", how= "left")
print (f"Merging on ARMAST.sono. \n Number of rows: {left_armast_s.shape[0]}\n")

right_artran_s = pd.merge(armast, artran, on ="sono", how= "right")
print (f"Merging on ARTRAN.sono. \n Number of rows: {right_artran_s.shape[0]}\n")

Outer Merge on invoice number. 
 Number of rows: 533942

Merging on ARMAST.invno. 
 Number of rows: 533883

Merging on ARTRAN.invno. 
 Number of rows: 528748

Outer Merge on sono number. 
 Number of rows: 608561

Merging on ARMAST.sono. 
 Number of rows: 608502

Merging on ARTRAN.sono. 
 Number of rows: 604000



In [0]:
#see how the nulls are distributed

#merge_outer_invo.isnull().sum() 
#left_armast.isnull().sum()
right_artran.isnull().sum()


invno                0
custno              59
invdate             59
amount              59
paid                59
status              59
duedate             59
days                59
sono_x              59
delivery            59
added               59
route             8007
seq              30205
salesman          5651
priccode            59
firstrun            59
diff_amt_paid       59
sono_y               0
invline              0
item                 1
desc                 0
uom                  1
units                0
ordqty               0
shipqty              0
price                0
totalcost            0
datesold             0
orgprice             0
diff_quantity        0
dtype: int64

In [0]:
right_artran= pd.merge(armast, artran, on ="invno", how= "right")
print (f"Merging on ARTRAN.invno. \n Number of rows: {right_artran.shape[0]}\n")

right_artran2= pd.merge(armast2, artran2, on ="invno", how= "right")
print (f"Merging on ARTRAN.invno. \n Number of rows: {right_artran.shape[0]}\n")

Merging on ARTRAN.invno. 
 Number of rows: 528748

Merging on ARTRAN.invno. 
 Number of rows: 528748



## 4.2 Cleaning Up the Merged Data

In [0]:
#getting rid of insignificant null values
right_artran.drop(columns= ['seq'], inplace=True) #because 30,146 null values
right_artran= right_artran.dropna(subset=['custno', 'item', 'uom']) #deleting less than .01 of data with only 64 entries

right_artran2.drop(columns= ['seq'], inplace=True) #because 30,146 null values
right_artran2= right_artran2.dropna(subset=['custno', 'item', 'uom']) #deleting less than .01 of data with only 64 entries

#Still need to figure out what to do with the "route" & "salesman" columns... There maybe another table to reference this and fill it in.



In [0]:
right_artran.isnull().sum()


invno               0
custno              0
invdate             0
amount              0
paid                0
status              0
duedate             0
days                0
sono_x              0
delivery            0
added               0
route            7948
salesman         5592
priccode            0
firstrun            0
diff_amt_paid       0
sono_y              0
invline             0
item                0
desc                0
uom                 0
units               0
ordqty              0
shipqty             0
price               0
totalcost           0
datesold            0
orgprice            0
diff_quantity       0
dtype: int64

In [0]:
#Adding a profit column
right_artran['profit'] = right_artran['amount'] - (right_artran['totalcost'] * right_artran['shipqty'])
right_artran.profit.describe()

count    528688.000000
mean        694.590718
std         675.015947
min       -7470.000000
25%         250.200000
50%         493.200000
75%         942.400000
max       20244.900000
Name: profit, dtype: float64

In [0]:
#Correct unit types
right_artran[['invno', 'sono_x', 'sono_y']]= right_artran[['invno', 'sono_x', 'sono_y']].astype('int64')
right_artran2[['invno', 'sono_x', 'sono_y']]= right_artran2[['invno', 'sono_x', 'sono_y']].astype('int64')

#reorder dataframe
right_artran = right_artran[['invdate','invno', 'item','desc','uom','units','price', 'totalcost', 'ordqty', 'shipqty','amount', 'custno', 
              'sono_x', 'sono_y', 'paid', 'duedate','days', 'status','datesold',
              'delivery', 'route', 'salesman', 'priccode','orgprice',
            'firstrun','added', 'invline']]

right_artran2 = right_artran2[['invdate','invno', 'item','desc','uom','units','price', 'totalcost', 'ordqty', 'shipqty','amount', 'custno', 
              'sono_x', 'sono_y', 'paid', 'duedate','days', 'status','datesold',
              'delivery', 'route', 'salesman', 'priccode','orgprice',
            'firstrun','added', 'invline']]

In [0]:
right_artran.nunique()

invdate             943
invno             67366
item               3088
desc               4096
uom                  41
units                41
price              1597
totalcost          1728
ordqty              447
shipqty             541
diff_quantity       286
amount            24777
profit           145546
custno              602
sono_x            60548
sono_y            60548
paid              24343
diff_amt_paid      1701
duedate             992
days                  9
status                3
datesold            943
delivery            943
route                17
salesman             15
priccode             18
orgprice            635
firstrun              4
added               967
invline             443
dtype: int64

In [0]:
# creating a daily_df
daily_df= right_artran.set_index('invdate').resample('D', level= 0).sum()
daily_df2= right_artran2.set_index('invdate').resample('D', level= 0).sum()

daily_df.head()

,invno,units,price,totalcost,ordqty,shipqty,amount,sono_x,sono_y,paid,days,orgprice,firstrun,invline
invdate,,,,,,,,,,,,,,
2017-01-02,9330544,45.0,1553.83,874.08,-36.91,-36.91,816.10,9386181,9386181,816.10,790.0,496.75,20.0,66
2017-01-03,130644691,1511.1,17720.93,11956.76,1131.00,1060.33,300649.29,133105237,133105237,300649.29,9313.0,17456.89,642.0,5100
2017-01-04,51236742,763.8,7115.52,4903.92,440.50,430.00,119299.31,52204904,52204904,119299.31,4213.0,7054.22,251.0,2021
2017-01-05,126977819,1190.3,20408.30,14078.24,947.50,875.20,476434.93,129398279,129398279,476434.93,10111.0,20580.61,619.0,6681
2017-01-06,130965995,1316.6,19107.14,12481.26,1622.59,1563.28,363886.59,133471194,133471194,363886.59,8555.0,18651.04,695.0,5539


In [0]:
dfwkly = right_artran.set_index('invdate').resample('7D').sum()
print (dfwkly.shape)
print (dfwkly.head())

(138, 17)
                invno   units     price  ...  orgprice  firstrun  invline
invdate                                  ...                             
2017-01-02  500644647  5354.8  73061.40  ...  71150.25    2475.0    21300
2017-01-09  670090064  7126.2  95147.75  ...  93182.21    3307.0    30098
2017-01-16  622873728  6448.9  89323.51  ...  87239.78    2999.0    34348
2017-01-23  595600421  6815.1  84489.71  ...  81575.88    2965.0    25781
2017-01-30  653324370  6813.0  93173.58  ...  91319.30    3159.0    32705

[5 rows x 17 columns]


In [0]:
dfmth = right_artran.set_index('invdate').resample('M').sum()
print (dfmth.shape)
print (dfmth.head())


(32, 17)
                 invno    units      price  ...   orgprice  firstrun  invline
invdate                                     ...                              
2017-01-31  2638169372  28360.2  377862.21  ...  367906.13   12946.0   124463
2017-02-28  2604083969  29089.2  374167.81  ...  363713.83   12594.0   112489
2017-03-31  2891062564  30786.3  419442.62  ...  408942.24   13816.0   117194
2017-04-30  2682505277  28630.6  393710.05  ...  385667.22   12620.0   112343
2017-05-31  2818316221  29889.6  415994.67  ...  401107.26   12977.0   130186

[5 rows x 17 columns]


In [0]:
df2wk = right_artran.set_index('invdate').resample('2W').sum()
print (df2wk.shape)
print (df2wk.head())

(70, 17)
                 invno    units      price  ...   orgprice  firstrun  invline
invdate                                     ...                              
2017-01-08   500644647   5354.8   73061.40  ...   71150.25    2475.0    21300
2017-01-22  1292963792  13575.1  184471.26  ...  180421.99    6306.0    64446
2017-02-05  1248924791  13628.1  177663.29  ...  172895.18    6124.0    58486
2017-02-19  1309279862  14543.9  192906.67  ...  187424.22    6343.0    56437
2017-03-05  1285698230  14356.3  181481.52  ...  175850.72    6173.0    53142

[5 rows x 17 columns]


## 4.3 Exporting cleaned up file

In [0]:
right_artran.reset_index(inplace= True)
daily_df.reset_index(inplace= True)

In [0]:
right_artran.head()

,index,invdate,invno,item,desc,uom,units,price,totalcost,ordqty,shipqty,diff_quantity,amount,profit,custno,sono_x,sono_y,paid,diff_amt_paid,duedate,days,status,datesold,delivery,route,salesman,priccode,orgprice,firstrun,added,invline
0,0,2017-01-02,207305,30172,"Carrot, Table, 25lb, Cert. OG",CASE,1.0,26.50,20.00,4.0,4.0,0.0,106.00,26.00,10-AP,211277,211277,106.00,0.0,2017-01-17,15.0,X,2017-01-02,2017-01-02,NaN,HS,PLAM,26.50,3.0,1/2/2017,1
1,1,2017-01-02,207306,40009C,"Tomato, 1 Layer 25ct 15lb, Certifie",CASE,1.0,49.90,34.00,1.0,1.0,0.0,49.90,15.90,30-ZEAL,211279,211279,49.90,0.0,2017-01-23,21.0,X,2017-01-02,2017-01-02,B1,EK,PL2I,49.90,1.0,1/2/2017,1
2,2,2017-01-02,207307,20081,"Kale, Lacinato 24ct, Certified Orga",CASE,1.0,30.90,24.00,1.0,1.0,0.0,30.90,6.90,10-AP,211280,211280,30.90,0.0,2017-01-17,15.0,X,2017-01-02,2017-01-02,NaN,HS,PLAM,30.90,3.0,1/2/2017,1
3,3,2017-01-02,207308,CV70187A,"Juice, Apple,1 gallon NATURAL, LOCA",EACH,1.0,4.90,3.75,4.0,4.0,0.0,167.65,152.65,30-SNFT,211285,211285,167.65,0.0,2017-01-16,14.0,X,2017-01-02,2017-01-02,FTC,EK,PL6,4.90,1.0,1/2/2017,1
4,4,2017-01-02,207308,G5103,"Jam, Blackberry 9 lb, Certified Org",EACH,1.0,49.35,39.50,1.0,1.0,0.0,167.65,128.15,30-SNFT,211285,211285,167.65,0.0,2017-01-16,14.0,X,2017-01-02,2017-01-02,FTC,EK,PL6,49.35,1.0,1/2/2017,2


In [0]:
right_artran.set_index('invdate', inplace=True)
right_artran2.set_index('invdate', inplace=True)
right_artran.head()


,invno,item,desc,uom,units,price,totalcost,ordqty,shipqty,amount,custno,sono_x,sono_y,paid,duedate,days,status,datesold,delivery,route,salesman,priccode,orgprice,firstrun,added,invline
invdate,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,207305,30172,"Carrot, Table, 25lb, Cert. OG",CASE,1.0,26.50,20.00,4.0,4.0,106.00,10-AP,211277,211277,106.00,2017-01-17,15.0,X,2017-01-02,2017-01-02,NaN,HS,PLAM,26.50,3.0,1/2/2017,1
2017-01-02,207306,40009C,"Tomato, 1 Layer 25ct 15lb, Certifie",CASE,1.0,49.90,34.00,1.0,1.0,49.90,30-ZEAL,211279,211279,49.90,2017-01-23,21.0,X,2017-01-02,2017-01-02,B1,EK,PL2I,49.90,1.0,1/2/2017,1
2017-01-02,207307,20081,"Kale, Lacinato 24ct, Certified Orga",CASE,1.0,30.90,24.00,1.0,1.0,30.90,10-AP,211280,211280,30.90,2017-01-17,15.0,X,2017-01-02,2017-01-02,NaN,HS,PLAM,30.90,3.0,1/2/2017,1
2017-01-02,207308,CV70187A,"Juice, Apple,1 gallon NATURAL, LOCA",EACH,1.0,4.90,3.75,4.0,4.0,167.65,30-SNFT,211285,211285,167.65,2017-01-16,14.0,X,2017-01-02,2017-01-02,FTC,EK,PL6,4.90,1.0,1/2/2017,1
2017-01-02,207308,G5103,"Jam, Blackberry 9 lb, Certified Org",EACH,1.0,49.35,39.50,1.0,1.0,167.65,30-SNFT,211285,211285,167.65,2017-01-16,14.0,X,2017-01-02,2017-01-02,FTC,EK,PL6,49.35,1.0,1/2/2017,2


In [0]:
df_2018 = right_artran ['2018':]
df_2018.reset_index(inplace= True)

In [0]:
df_2018.head()

,invdate,index,invno,item,desc,uom,units,price,totalcost,ordqty,shipqty,diff_quantity,amount,profit,custno,sono_x,sono_y,paid,diff_amt_paid,duedate,days,status,datesold,delivery,route,salesman,priccode,orgprice,firstrun,added,invline
0,2018-01-02,167166,231007,10018,"Apple, Fuji 100 ct WXF, Cert. OG",CS40,1.0,56.9,40.0,1.0,0.0,1.0,2048.1,2048.1,20-VC,236964,236964,2048.1,0.0,2018-01-23,21.0,X,2018-01-02,2018-01-02,FTC,NC,PL2I,56.9,1.0,12/30/2017,2
1,2018-01-02,167167,231007,10090A,"Apple, Sweet Orin, 100ct, Cert OG",CASE,1.0,52.9,36.0,1.0,1.0,0.0,2048.1,2012.1,20-VC,236964,236964,2048.1,0.0,2018-01-23,21.0,X,2018-01-02,2018-01-02,FTC,NC,PL2I,52.9,1.0,12/30/2017,3
2,2018-01-02,167168,231007,10313P,"Grapefruit, Rio Red 36ct, Certified",CASE,1.0,28.9,20.0,4.0,4.0,0.0,2048.1,1968.1,20-VC,236964,236964,2048.1,0.0,2018-01-23,21.0,X,2018-01-02,2018-01-02,FTC,NC,PL2I,28.9,1.0,12/30/2017,4
3,2018-01-02,167169,231007,10334,"Lemon, 95ct, Fancy, 38lb, Cert. OG",CS38,1.0,62.9,45.0,1.0,1.0,0.0,2048.1,2003.1,20-VC,236964,236964,2048.1,0.0,2018-01-23,21.0,X,2018-01-02,2018-01-02,FTC,NC,PL2I,62.9,1.0,12/30/2017,5
4,2018-01-02,167170,231007,10336-A,"Lemon, Meyer 25lb, Certified Organi",CASE,1.0,40.9,28.0,1.0,1.0,0.0,2048.1,2020.1,20-VC,236964,236964,2048.1,0.0,2018-01-23,21.0,X,2018-01-02,2018-01-02,FTC,NC,PL2I,40.9,1.0,12/30/2017,7


In [0]:
#creating one and two year data frames

one_year= right_artran ['2018-08-23': '2019-08-23']
new_info = right_artran2 ['2019-08-24': ]
#two_year= right_artran ['2017-08-23': '2019-08-23']

In [0]:
new_info.head()

,invno,item,desc,uom,units,price,totalcost,ordqty,shipqty,amount,custno,sono_x,sono_y,paid,duedate,days,status,datesold,delivery,route,salesman,priccode,orgprice,firstrun,added,invline
invdate,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-08-24,279901,CV30233E,Cucumber English CV,CASE,1.0,20.0,40.0,1.0,1.0,20.0,30-KTD,288203,288203,20.0,2019-09-23,30,X,2019-08-24,2019-08-24,D2,AR,PL6,58.9,1,2019-08-24 09:06:25,117
2019-08-24,279902,CV30490,"Peas, Snow Peas, CV",CASE,1.0,40.9,42.0,1.0,1.0,40.9,30-EAH,288204,288204,40.9,2019-08-24,0,X,2019-08-24,2019-08-24,BG1,AR,PL6,61.9,1,2019-08-24 09:37:05,3
2019-08-24,279903,60005A,"Mushroom, Button, 12 x 8oz, Certified Organic",CASE,1.0,23.9,17.0,1.0,1.0,23.9,20-ALFL,288205,288205,23.9,2019-09-14,21,X,2019-08-24,2019-08-24,D3,NC,PL2I,23.9,1,2019-08-24 09:59:42,245
2019-08-24,279854,20081,"Kale, Lacinato 24ct, Certified Organic",CASE,1.0,31.9,22.0,1.0,1.0,31.9,20-ALFL,288141,288141,31.9,2019-09-14,21,X,2019-08-24,2019-08-24,BG1,NC,PL2I,31.9,1,2019-08-23 09:55:18,1
2019-08-24,279856,40071,"Tomato, Heirloom Mix, 10lb, Cert. O",CASE,1.0,24.5,20.0,4.0,4.0,1248.0,20-ALF,288147,288147,1248.0,2019-09-14,21,X,2019-08-24,2019-08-24,BG1,NC,PL2I,28.9,1,2019-08-23 12:02:06,1


In [0]:
master = pd.concat([one_year, new_info])
master['2019-08-23':'2019-08-24']

,invno,item,desc,uom,units,price,totalcost,ordqty,shipqty,amount,custno,sono_x,sono_y,paid,duedate,days,status,datesold,delivery,route,salesman,priccode,orgprice,firstrun,added,invline
invdate,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-08-23,279579,90022E,"Flour, Coconut 50lb, Certified Organic",CASE,1.0,95.9,69.0,1.0,1.0,384.6,80-WL,287901,287901,0.0,2019-08-23,0.0,O,2019-08-23,2019-08-23,D3,NC,PL7,95.9,1.0,8/21/2019,1
2019-08-23,279579,90011A,"Flour, Garbanzo, 50lb., Cert. OG, USA",EACH,1.0,115.9,83.0,1.0,1.0,384.6,80-WL,287901,287901,0.0,2019-08-23,0.0,O,2019-08-23,2019-08-23,D3,NC,PL7,115.9,1.0,8/21/2019,2
2019-08-23,279579,90048,"Oats Rolled, 50 LB, LOCAL, Cert. OG, USA",CASE,1.0,56.5,48.0,1.0,1.0,384.6,80-WL,287901,287901,0.0,2019-08-23,0.0,O,2019-08-23,2019-08-23,D3,NC,PL7,56.5,1.0,8/21/2019,3
2019-08-23,279579,50137,"Yam, Jewel, Jumbo, 40lb, Cert. OG",CASE,1.0,61.9,38.0,1.0,1.0,384.6,80-WL,287901,287901,0.0,2019-08-23,0.0,O,2019-08-23,2019-08-23,D3,NC,PL7,61.9,1.0,8/21/2019,4
2019-08-23,279579,30762,"Squash, Zucchini, Medium, 22lb, Cert. OG",CASE,1.0,31.9,19.0,1.0,1.0,384.6,80-WL,287901,287901,0.0,2019-08-23,0.0,O,2019-08-23,2019-08-23,D3,NC,PL7,31.9,1.0,8/21/2019,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-24,279825,60028,"Mushroom, Porcini, 1 lb., Wild Crafted",1LB,1.0,31.9,21.0,2.0,2.0,992.4,30-CLURO,288144,288144,992.4,2019-09-14,21.0,X,2019-08-24,2019-08-24,GC,AR,PL6,31.9,1.0,2019-08-23 10:30:14,31
2019-08-24,279825,20384,"Fennel, 12ct, Cert. OG",CASE,1.0,25.9,17.0,1.0,1.0,992.4,30-CLURO,288144,288144,992.4,2019-09-14,21.0,X,2019-08-24,2019-08-24,GC,AR,PL6,25.9,1.0,2019-08-23 10:30:14,32
2019-08-24,279825,90021,"Flour,Unbleached White, (AP) 50LB,CertOG,USA",CASE,1.0,46.0,39.0,1.0,1.0,992.4,30-CLURO,288144,288144,992.4,2019-09-14,21.0,X,2019-08-24,2019-08-24,GC,AR,PL6,46.0,1.0,2019-08-23 10:30:14,33


In [0]:
one_year.reset_index(inplace=True)
new_info.reset_index(inplace=True)
master.reset_index(inplace=True)


#two_year.reset_index(inplace=True)

In [0]:
 #exporting the weekly data
from google.colab import files
# right_artran.to_csv(r'/content/drive/My Drive/Capstone_2/data/df_wkly.csv', index = None, header=True)
# daily_df.to_csv(r'/content/drive/My Drive/Capstone_2/data/df_daily2.csv', index = None, header=True)
#df_2018.to_csv(r'/content/drive/My Drive/Capstone_2/data/df_2018.csv', index = None, header=True)
one_year.to_csv(r'/content/drive/My Drive/Capstone_2/data/one_year.csv', index = None, header=True)
new_info.to_csv(r'/content/drive/My Drive/Capstone_2/data/new_info.csv', index = None, header=True)
master.to_csv(r'/content/drive/My Drive/Capstone_2/data/master.csv', index = None, header=True)


#two_year.to_csv(r'/content/drive/My Drive/Capstone_2/data/two_year.csv', index = None, header=True)

In [0]:
right_artran.shape

(528688, 30)

In [0]:
#Final inspection of the columns and values of the clean dataset
col_descriptors(right_artran)



Column Descriptions: 

index column:
Data Type: int64
Unique Vals: 528688
Nulls: 0 

invno column:
Data Type: int64
Unique Vals: 67366
Nulls: 0 

item column:
Data Type: object
Unique Vals: 3088
Nulls: 0 

desc column:
Data Type: object
Unique Vals: 4096
Nulls: 0 

uom column:
Data Type: object
Unique Vals: 41
Nulls: 0 

units column:
Data Type: float64
Unique Vals: 41
Nulls: 0 

price column:
Data Type: float64
Unique Vals: 1597
Nulls: 0 

totalcost column:
Data Type: float64
Unique Vals: 1728
Nulls: 0 

ordqty column:
Data Type: float64
Unique Vals: 447
Nulls: 0 

shipqty column:
Data Type: float64
Unique Vals: 541
Nulls: 0 

diff_quantity column:
Data Type: float64
Unique Vals: 286
Nulls: 0 

amount column:
Data Type: float64
Unique Vals: 24777
Nulls: 0 

profit column:
Data Type: float64
Unique Vals: 145546
Nulls: 0 

custno column:
Data Type: object
Unique Vals: 602
Nulls: 0 

sono_x column:
Data Type: int64
Unique Vals: 60548
Nulls: 0 

sono_y column:
Data Type: int64
Unique Vals